In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2023-03-31 16:10:21.040631: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.__version__

'2.11.0'

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
df = pd.read_csv('../datasets/churn.csv', index_col='RowNumber')

In [4]:
df.drop(columns=['CustomerId', 'Surname'], inplace=True)

In [5]:
df.describe()[1:]

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
mean,650.53,38.92,5.01,"76,485.89",1.53,0.71,0.52,"100,090.24",0.20
std,96.65,10.49,2.89,"62,397.41",0.58,0.46,0.50,"57,510.49",0.40
min,350.00,18.00,0.00,0.00,1.00,0.00,0.00,11.58,0.00
25%,584.00,32.00,3.00,0.00,1.00,0.00,0.00,"51,002.11",0.00
50%,652.00,37.00,5.00,"97,198.54",1.00,1.00,1.00,"100,193.91",0.00
75%,718.00,44.00,7.00,"127,644.24",2.00,1.00,1.00,"149,388.25",0.00
max,850.00,92.00,10.00,"250,898.09",4.00,1.00,1.00,"199,992.48",1.00


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1 to 10000
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 937.5+ KB


In [7]:
df.Geography.unique(), df.Gender.unique()

(array(['France', 'Spain', 'Germany'], dtype=object),
 array(['Female', 'Male'], dtype=object))

In [8]:
df.Geography = pd.Categorical(df.Geography)
df.Gender = pd.Categorical(df.Gender)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1 to 10000
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   CreditScore      10000 non-null  int64   
 1   Geography        10000 non-null  category
 2   Gender           10000 non-null  category
 3   Age              10000 non-null  int64   
 4   Tenure           10000 non-null  int64   
 5   Balance          10000 non-null  float64 
 6   NumOfProducts    10000 non-null  int64   
 7   HasCrCard        10000 non-null  int64   
 8   IsActiveMember   10000 non-null  int64   
 9   EstimatedSalary  10000 non-null  float64 
 10  Exited           10000 non-null  int64   
dtypes: category(2), float64(2), int64(7)
memory usage: 801.0 KB


In [10]:
df = df[['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 
                  'Geography', 'Gender', 'Exited']]

In [11]:
df.head(1)

,CreditScore,Age,Tenure,Balance,EstimatedSalary,NumOfProducts,HasCrCard,IsActiveMember,Geography,Gender,Exited
RowNumber,,,,,,,,,,,
1,619,42,2,0.00,"101,348.88",1,1,1,France,Female,1


In [12]:
for col in ['Age', 'Tenure', 'HasCrCard', 'HasCrCard', 'NumOfProducts', 'IsActiveMember', 'Exited']:
    df[col] = df[col].astype('uint8')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1 to 10000
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   CreditScore      10000 non-null  int64   
 1   Age              10000 non-null  uint8   
 2   Tenure           10000 non-null  uint8   
 3   Balance          10000 non-null  float64 
 4   EstimatedSalary  10000 non-null  float64 
 5   NumOfProducts    10000 non-null  uint8   
 6   HasCrCard        10000 non-null  uint8   
 7   IsActiveMember   10000 non-null  uint8   
 8   Geography        10000 non-null  category
 9   Gender           10000 non-null  category
 10  Exited           10000 non-null  uint8   
dtypes: category(2), float64(2), int64(1), uint8(6)
memory usage: 390.9 KB


In [14]:
df.Geography.nunique(), df.Geography.unique()

(3,
 ['France', 'Spain', 'Germany']
 Categories (3, object): ['France', 'Germany', 'Spain'])

In [15]:
np.iinfo('uint8')

iinfo(min=0, max=255, dtype=uint8)

In [16]:
df.iloc[:, :-1].head(1)

,CreditScore,Age,Tenure,Balance,EstimatedSalary,NumOfProducts,HasCrCard,IsActiveMember,Geography,Gender
RowNumber,,,,,,,,,,
1,619,42,2,0.00,"101,348.88",1,1,1,France,Female


In [17]:
X = df.iloc[:,:-1].values
y = df.iloc[:, -1].values

In [18]:
type(X), type(y)

(numpy.ndarray, numpy.ndarray)

In [19]:
X[:2]

array([[619, 42, 2, 0.0, 101348.88, 1, 1, 1, 'France', 'Female'],
       [608, 41, 1, 83807.86, 112542.58, 1, 0, 1, 'Spain', 'Female']],
      dtype=object)

In [18]:
X[:, -2:-1] # categorical values to be transformed

array([['France'],
       ['Spain'],
       ['France'],
       ...,
       ['France'],
       ['Germany'],
       ['France']], dtype=object)

Split the data

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 2912)

In [43]:
# encode categorical variables
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
le = LabelEncoder()
ohe = OneHotEncoder()

In [44]:
# Gender
X_train[:, -1] = le.fit_transform(X_train[:, -1])
X_test[:, -1] = le.transform(X_test[:, -1])

In [45]:
X_test[:2]

array([[661, 41, 5, 28082.95, 69586.27, 1, 1, 0, 'Spain', 0],
       [521, 22, 5, 0.0, 99828.45, 2, 1, 1, 'France', 1]], dtype=object)

In [47]:
len(X_train), len(X_train[0])

(7500, 10)

In [48]:
ct = ColumnTransformer(transformers=[('one hot encoder', ohe, [-1]), # didn't accept label encoder
                                    ('scaler', StandardScaler(), [0,1,2, 3, 4])], remainder='passthrough') 
# default remainder = 'drop', columns that are not specified are dropped

In [49]:
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [50]:
len(X_train[0])

11

Label encoding is better to apply only to the binary categories (Male/Female) or to the `y` values
For ANN we need to __scale ALL values, including 0 and 1__. Redo everything. This time I'll use One Hot Encoder for both columns and I'll add additional parameters: `drop='first'` (by default encodes everything) and `dtype='uint8'` (by default returns float values)

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 2912)

In [57]:
len(X_train), len(X_train[0]) # 7500 rows, 10 columns

(7500, 10)

In [64]:
ohe1 = OneHotEncoder(categories='auto', drop='first', sparse_output=True, dtype='uint8')

In [65]:
ct = ColumnTransformer(transformers=[('one hot encoder', ohe1, [-2, -1])], # didn't accept label encoder
                                   remainder='passthrough')
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [66]:
len(X_train), len(X_train[0]) 

(7500, 11)

In [68]:
X_train[0]

array([0, 0, 1, 578, 30, 4, 169462.09, 112187.11, 1, 1, 0], dtype=object)

In [69]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [70]:
X_train[0]

array([-0.57406576, -0.57098639,  0.90966223, -0.74782002, -0.84933043,
       -0.35447986,  1.49170097,  0.20189918, -0.90804738,  0.6461435 ,
       -1.03308019])

#### Building ANN

In [71]:
ann = tf.keras.models.Sequential()

2023-03-31 21:00:29.727072: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [72]:
# add hidden layers
ann.add(tf.keras.layers.Dense(units=7, activation='relu'))
ann.add(tf.keras.layers.Dense(units=5, activation='relu'))
ann.add(tf.keras.layers.Dense(units=7, activation='relu'))

# add an output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [73]:
# compile ANN, adam-> for stochastic GD, loos for non binary _-> catogory_crossentropy
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [75]:
# train the model
ann.fit(X_train, y_train, batch_size=32, epochs=50, verbose=None) # no big changes after 50 and goes under 0.86 after 100

Epoch 1/50
235/235 [==============================] - 0s 849us/step - loss: 0.3212 - accuracy: 0.8669
Epoch 2/50
235/235 [==============================] - 0s 848us/step - loss: 0.3209 - accuracy: 0.8660
Epoch 3/50
235/235 [==============================] - 0s 851us/step - loss: 0.3212 - accuracy: 0.8672
Epoch 4/50
235/235 [==============================] - 0s 910us/step - loss: 0.3211 - accuracy: 0.8668
Epoch 5/50
235/235 [==============================] - 0s 893us/step - loss: 0.3215 - accuracy: 0.8669
Epoch 6/50
235/235 [==============================] - 0s 882us/step - loss: 0.3206 - accuracy: 0.8669
Epoch 7/50
235/235 [==============================] - 0s 878us/step - loss: 0.3212 - accuracy: 0.8676
Epoch 8/50
235/235 [==============================] - 0s 872us/step - loss: 0.3204 - accuracy: 0.8672
Epoch 9/50
235/235 [==============================] - 0s 811us/step - loss: 0.3209 - accuracy: 0.8651
Epoch 10/50
235/235 [==============================] - 0s 902us/step - loss: 0.321

In [76]:
# repeat fit without printing epochs
ann.fit(X_train, y_train, batch_size=32, epochs=50, verbose=None) # skip printing epochs

In [78]:
ann.predict(X_test) # returns a probability of churn because we picked 'sigmoid' as an activation function

79/79 [==============================] - 0s 623us/step


array([[0.53509086],
       [0.00421836],
       [0.3524722 ],
       ...,
       [0.06565866],
       [0.0761841 ],
       [0.1573591 ]], dtype=float32)

In [79]:
ann.predict(X_test) > 0.5

79/79 [==============================] - 0s 610us/step


array([[ True],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [80]:
y_pred = ann.predict(X_test)

79/79 [==============================] - 0s 603us/step


In [81]:
type(y_pred)

numpy.ndarray

In [82]:
y_pred = np.where((y_pred > 0.5), 1, 0)

In [83]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [88]:
len(y_pred) == len(y_test)

True

In [89]:
y_pred.shape, y_test.shape

((2500, 1), (2500,))

In [95]:
np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),axis=1)

array([[1, 1],
       [0, 0],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]])

In [96]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.852